In [1]:
# import 및 폰트 설정
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
# import sys
# !{sys.executable} -m pip install seaborn
import seaborn as sns
fname='./NanumBarunGothic.otf'
font = fm.FontProperties(fname = fname).get_name()
plt.rcParams["font.family"] = font

# OPS 예측 할 선수
targets = [76232, 68050, 75847, 67341, 79192, 78224, 78513, 76290, 79215, 67872]

years = ['2018', '2019', '2020', '2021']
hitter_basic = pd.DataFrame({})
hts = pd.DataFrame({})
hitter_info = pd.DataFrame({})

hitter_info_base = "./2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_"
hts_base = "./2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_"
hitter_basic_base = "./2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_"

for year in years:
    hitter_basic_file = hitter_basic_base + year + ".csv"
    hitter_info_file = hitter_info_base + year + ".csv"
    hts_file = hts_base + year + ".csv"
    # ignore_index를 통해 각자의 index를 없애고, 합쳐서 index 구성
    hitter_basic = pd.concat([hitter_basic, pd.read_csv(hitter_basic_file, encoding='cp949')], axis=0, ignore_index=True) 
    hts = pd.concat([hts, pd.read_csv(hts_file, encoding='cp949')], axis=0, ignore_index=True) 
    hitter_info = pd.concat([hitter_info, pd.read_csv(hitter_info_file, encoding='cp949')], axis=0, ignore_index=True) 


#hitter_info, hts
hts

,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
0,2018,20180324HHWO0,180324_140436,62797,HH,1,131.50,42.7,플라이,149.59,고척
1,2018,20180324HHWO0,180324_140514,76753,HH,1,135.18,9.9,1루타,148.78,고척
2,2018,20180324HHWO0,180324_140647,71752,HH,1,152.41,2.1,1루타,148.59,고척
3,2018,20180324HHWO0,180324_140911,62700,HH,1,113.72,13.5,1루타,139.13,고척
4,2018,20180324HHWO0,180324_142050,68730,HH,2,54.11,16.4,번트안타,134.07,고척
...,...,...,...,...,...,...,...,...,...,...,...
120740,2021,20210711LTSS0,210711_195321,50458,SS,8,141.23,-4.9,땅볼아웃,137.76,대구
120741,2021,20210711LTSS0,210711_195424,62415,SS,8,101.09,31.8,1루타,138.51,대구
120742,2021,20210711LTSS0,210711_195627,75566,SS,8,146.84,33.4,홈런,134.10,대구
120743,2021,20210711LTSS0,210711_200047,69418,SS,8,146.12,23.9,홈런,139.96,대구


문제 조건에 의해 barrel 타구를 기준으로 예측 모형을 개발해야 한다.

시기별 차이 9월 이전까지의 barrel 타구 비율과 9월 이후의 barrel 타구 비율을 구한다.

이를 통해, 각 선수들이 시기에 따라 가지는 변화를 예측해, 올해 9월 이전의 결과를 활용하여 9월 예측한다.

--> barrel 타구가 아닌 경우에 대해서는, barrel 타구의 타율 장타율과 전체 타구의 타율, 장타율 비교를 통해 추측


In [76]:
# 위 방법 틀림 -> hts 전체 중, barrel 타구 비율 구해야 하는데, barrel 타구들에 대한 타율, 장타율 구했다.
# 1. hts에서 각 선수의 데이터를 뽑아온다.
# 2. 그 선수들의 타구 중 barrel 타구 기준에 해당하는 것들의 개수를 구한다.
# 3. 그렇게 해서, 전체 타구 중 barrel 타구의 비율을 구한다.
# 4. 이후, 9월 이후의 barrel 타구 비율도 구해, 선수마다 9월 이후가 강한 선수와 약한 선수를 구분학자 한다.

# 1. hts
# 2. 각 선수들의 barrel 타구 개수
targets = [76232, 68050, 75847, 67341, 79192, 78224, 78513, 76290, 79215, 67872]
total_of_target = []
barrel_of_target = []
years = ['2018', '2019', '2020']
barrel_on_base_of_target = []
barrel_slugging_of_target = []
hits = ['', '1루타', '2루타', '3루타', '홈런']
barrel_vel = [138, 142, 147, 151, 154, 167, 172]
barrel_ang = [(23, 29), (21, 33), (20, 36), (19, 37), (18, 40), (15, 45), (9, 51)]
for target in targets:
    target_hts = hts[hts['PCODE'] == target]
    target_hts.drop_duplicates()
    total_of_target.append(len(target_hts))
    target_barrel = pd.DataFrame({})
    for i in range(len(barrel_vel)):
        target_barrel = pd.concat([target_barrel, target_hts[(target_hts['HIT_VEL'] >= barrel_vel[i]) &\
                                                              ((target_hts['HIT_ANG_VER']>=barrel_ang[i][0]) & \
                                                               (target_hts['HIT_ANG_VER']<=barrel_ang[i][1]))]], axis = 0, ignore_index = True)
    target_barrel.drop_duplicates()
    target_hts.drop_duplicates()
    barrel_of_target.append(len(target_barrel))    
    
    
    total = len(target_barrel)
    base1 = len(target_barrel[target_barrel['HIT_RESULT']==hits[1]])
    base2 = len(target_barrel[target_barrel['HIT_RESULT']==hits[2]])
    base3 = len(target_barrel[target_barrel['HIT_RESULT']==hits[3]])
    homerun = len(target_barrel[target_barrel['HIT_RESULT']==hits[4]])
    barrels_on_base = (base1 + base2 + base3 + homerun) / total
    barrels_slugging = (base1 + (base2*2) + (base3*3) + (homerun*4)) / total
    barrel_on_base_of_target.append(barrels_on_base)
    barrel_slugging_of_target.append(barrels_slugging)
    
# 3. 9월 이후의 barrel 타구 비율(~2020 년 까지만, 2020년 9월 이후 데이터는 아직 X라 가정)

total_of_target_aft_9 = []
barrel_of_target_aft_9 = []
for target in targets:
    # 9월 이후만 
    target_total = pd.DataFrame({})
    target_barrel = pd.DataFrame({})
    for year in years:
        #print(year+'09')
        tmp = hts[(hts['PCODE'] == target)]
        tmp2 = tmp[tmp['G_ID'] > year]
        tmp3 = tmp2[tmp2['G_ID'] < str(int(year) + 1)]
        tmp4 = tmp3[tmp3['G_ID'] > (year+'09')]
        tmp4.drop_duplicates()
        #print(tmp4)
        target_total = pd.concat([target_total, tmp4])
    target_total.drop_duplicates()
    target_barrel.drop_duplicates()
    total_of_target_aft_9.append(len(target_total))
    for i in range(len(barrel_vel)):
        target_barrel = pd.concat([target_barrel, target_total[(target_total['HIT_VEL'] >= barrel_vel[i]) &\
                                                              ((target_total['HIT_ANG_VER']>=barrel_ang[i][0]) & \
                                                               (target_total['HIT_ANG_VER']<=barrel_ang[i][1]))]], axis = 0, ignore_index = True)
    target_barrel.drop_duplicates()
    barrel_of_target_aft_9.append(len(target_barrel))

total_of_target_bef_9 = []
barrel_of_target_bef_9 = []
for i in range(len(total_of_target)):
    total_of_target_bef_9.append(total_of_target[i] - total_of_target_aft_9[i])
    barrel_of_target_bef_9.append(barrel_of_target[i] - barrel_of_target_aft_9[i])

total_of_target, barrel_of_target, total_of_target_aft_9, barrel_of_target_aft_9, total_of_target_bef_9, barrel_of_target_bef_9
print("각 선수들의 9월 후/전 barrel 타구 비율의 비율")
barrel_ratio = []
barrel_ratio_bef_Sep = []
barrel_ratio_aft_Sep = []
for i in range(len(targets)):
#     print("선수 코드 : ", targets[i])
    a = barrel_of_target[i] / total_of_target[i]
    b = barrel_of_target_bef_9[i] / total_of_target_bef_9[i]
    c = barrel_of_target_aft_9[i] / total_of_target_aft_9[i]
    barrel_ratio.append(a)
    barrel_ratio_bef_Sep.append(b)
    barrel_ratio_aft_Sep.append(c)    
#     print("전체 barrel 타구 비율 : ", barrel_of_target[i] / total_of_target[i])
#     print("9월 전의 barrel 타구 비율 : ", barrel_of_target_bef_9[i] / total_of_target_bef_9[i])
#     print("9월부터 barrel 타구 비율 : ", barrel_of_target_aft_9[i] / total_of_target_aft_9[i])

barrel_ratio, barrel_ratio_bef_Sep, barrel_ratio_aft_Sep

bef_aft_Sep_ratio = []
for i in range(len(targets)):
    bef_aft_Sep_ratio.append(barrel_ratio_aft_Sep[i] / barrel_ratio_bef_Sep[i])
bef_aft_Sep_ratio


각 선수들의 9월 후/전 barrel 타구 비율의 비율


[1.3042281098941442,
 1.1365914786967417,
 0.8934528254916606,
 0.6816803699897226,
 0.9795771609377246,
 1.1849857796839591,
 0.9663414634146342,
 0.5134168157423972,
 0.7224294438026077,
 1.118073161897863]

In [3]:
# 2021년 타구 중 barrel 타구의 비율
total_of_target_2021 = []
barrel_of_target_2021 = []
for target in targets:
    # 9월 이후만 
    #target_total = pd.DataFrame({})
    #target_barrel = pd.DataFrame({})
    #print(year+'09')
    tmp = hts[(hts['PCODE'] == target)]
    tmp2 = tmp[tmp['G_ID'] > '2021']
    tmp2.drop_duplicates()
    total_of_target_2021.append(len(tmp2))
    
    target_barrel = pd.DataFrame({})
    for i in range(len(barrel_vel)):
        target_barrel = pd.concat([target_barrel, tmp2[(tmp2['HIT_VEL'] >= barrel_vel[i]) &\
                                                              ((tmp2['HIT_ANG_VER']>=barrel_ang[i][0]) & \
                                                               (tmp2['HIT_ANG_VER']<=barrel_ang[i][1]))]], axis = 0, ignore_index = True)
    target_barrel.drop_duplicates()
    barrel_of_target_2021.append(len(target_barrel))
total_of_target_2021, barrel_of_target_2021
barrel_ratio_2021 = []
for i in range(len(targets)):
    barrel_ratio_2021.append(barrel_of_target_2021[i]/total_of_target_2021[i])

print("각 선수들의 2021년 9월 전 barrel 타구 비율")
barrel_ratio_2021

각 선수들의 2021년 9월 전 barrel 타구 비율


[0.40865384615384615,
 0.25925925925925924,
 0.5136612021857924,
 0.17886178861788618,
 0.5116279069767442,
 0.5384615384615384,
 0.26222222222222225,
 0.5388349514563107,
 0.2670807453416149,
 0.485]

In [77]:
print("각 선수들의 예측 기간 barrel 타구 비율 예측 결과")

expected_barrel_ratio = []
for i in range(len(targets)):
    expected_barrel_ratio.append(barrel_ratio_2021[i] * bef_aft_Sep_ratio[i])
expected_barrel_ratio

각 선수들의 예측 기간 barrel 타구 비율 예측 결과


[0.5329778333702031,
 0.2946718648473034,
 0.45893205243833934,
 0.1219265702420642,
 0.5011790125727894,
 0.6380692659836703,
 0.25339620596205964,
 0.27664692498740817,
 0.19294699430752876,
 0.5422654835204636]

In [5]:
print("각 선수들의 전체 기간 barrel 타구 비율")


total_of_target, barrel_of_target
target_barrel_ratio = []
for i in range(len(targets)):
    target_barrel_ratio.append(barrel_of_target[i]/total_of_target[i])
target_barrel_ratio

각 선수들의 전체 기간 barrel 타구 비율


[0.42725880551301687,
 0.41602728047740833,
 0.5171232876712328,
 0.2120211360634082,
 0.41886151231945623,
 0.6555839727195226,
 0.35095137420718814,
 0.43859649122807015,
 0.3732394366197183,
 0.4984126984126984]

In [110]:
print("각 선수들의 전체 기간 출루율, 장타율")

target_on_base = []
target_slugging = []
for target in targets:
    tmp = hitter_basic[hitter_basic['PCODE'] == target]
    tmp['출루율*타수'] = tmp['AB'] * tmp['BA']
    tmp['장타율*타수'] = tmp['AB'] * tmp['SLG']
    tmp.drop_duplicates()
    target_on_base.append( tmp['출루율*타수'].sum() / tmp['AB'].sum() )
    target_slugging.append( tmp['장타율*타수'].sum() / tmp['AB'].sum() )
target_on_base, target_slugging

각 선수들의 전체 기간 출루율, 장타율


c:\users\lgy39\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\lgy39\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lgy39\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

([0.34638311688311685,
  0.32951785714285714,
  0.2746810237203496,
  0.3412071542979178,
  0.3147176829268292,
  0.2916093573844419,
  0.31037781186094066,
  0.32450808700501954,
  0.3191303820497271,
  0.2837873051224944],
 [0.6004272727272727,
  0.5310293778801843,
  0.5433414481897628,
  0.4883240790176188,
  0.4889201219512195,
  0.5275428410372042,
  0.5077464212678937,
  0.5062554378137201,
  0.4655051546391753,
  0.5368145879732739])

In [111]:
print(targets)
hitter_data = pd.DataFrame({})
hitter_data['출루율'] = target_on_base
hitter_data['장타율'] = target_slugging
hitter_data['전체 타수'] = target_count
hitter_data['barrel 타수'] = barrel_of_target
hitter_data['barrel ratio'] = hitter_data['barrel 타수'] / hitter_data['전체 타수']
hitter_data['barrel ratio 당 출루율'] = hitter_data['출루율'] / hitter_data['barrel ratio']
hitter_data['barrel ratio 당 장타율'] = hitter_data['장타율'] / hitter_data['barrel ratio']
hitter_data['barrel ratio의 9월 후/전 ratio'] = bef_aft_Sep_ratio
hitter_data['2021년 9월 전 barrel ratio'] = barrel_ratio_2021
hitter_data['2021년 9월 후 barrel ratio(예측)'] = hitter_data['barrel ratio의 9월 후/전 ratio']*hitter_data['2021년 9월 전 barrel ratio']
hitter_data['출루율(예측)'] = hitter_data['barrel ratio 당 출루율'] * hitter_data['2021년 9월 후 barrel ratio(예측)']
hitter_data['장타율(예측)'] = hitter_data['barrel ratio 당 장타율'] * hitter_data['2021년 9월 후 barrel ratio(예측)']
hitter_data['OPS(예측)'] = hitter_data['출루율(예측)'] + hitter_data['장타율(예측)']
hitter_data

[76232, 68050, 75847, 67341, 79192, 78224, 78513, 76290, 79215, 67872]


,출루율,장타율,전체 타수,barrel 타수,barrel ratio,barrel ratio 당 출루율,barrel ratio 당 장타율,barrel ratio의 9월 후/전 ratio,2021년 9월 전 barrel ratio,2021년 9월 후 barrel ratio(예측),출루율(예측),장타율(예측),OPS(예측)
0,0.346383,0.600427,1290,558,0.432558,0.800778,1.388085,1.304228,0.408654,0.532978,0.426797,0.739818,1.166615
1,0.329518,0.531029,1465,488,0.333106,0.989229,1.594176,1.136591,0.259259,0.294672,0.291498,0.469759,0.761257
2,0.274681,0.543341,1361,604,0.443791,0.618942,1.224317,0.893453,0.513661,0.458932,0.284052,0.561878,0.845931
3,0.341207,0.488324,1577,321,0.203551,1.676273,2.399025,0.681680,0.178862,0.121927,0.204382,0.292505,0.496887
4,0.314718,0.488920,1415,493,0.348410,0.903297,1.403290,0.979577,0.511628,0.501179,0.452714,0.703299,1.156013
5,0.291609,0.527543,1538,769,0.500000,0.583219,1.055086,1.184986,0.538462,0.638069,0.372134,0.673218,1.045352
6,0.310378,0.507746,1663,498,0.299459,1.036462,1.695547,0.966341,0.262222,0.253396,0.262636,0.429645,0.692281
7,0.324508,0.506255,1526,600,0.393185,0.825332,1.287576,0.513417,0.538835,0.276647,0.228326,0.356204,0.584530
8,0.319130,0.465505,1433,477,0.332868,0.958729,1.398467,0.722429,0.267081,0.192947,0.184984,0.269830,0.454814
9,0.283787,0.536815,1517,628,0.413975,0.685518,1.296732,1.118073,0.485000,0.542265,0.371733,0.703173,1.074906


In [113]:
tmp = pd.DataFrame({})
tmp['PCODE'] = targets
tmp['출루율'] = hitter_data['출루율(예측)']
tmp['장타율'] = hitter_data['장타율(예측)']
tmp['OPS'] = hitter_data['OPS(예측)']
tmp

,PCODE,출루율,장타율,OPS
0,76232,0.426797,0.739818,1.166615
1,68050,0.291498,0.469759,0.761257
2,75847,0.284052,0.561878,0.845931
3,67341,0.204382,0.292505,0.496887
4,79192,0.452714,0.703299,1.156013
5,78224,0.372134,0.673218,1.045352
6,78513,0.262636,0.429645,0.692281
7,76290,0.228326,0.356204,0.584530
8,79215,0.184984,0.269830,0.454814
9,67872,0.371733,0.703173,1.074906
